In [1]:
import sys
sys.path.append("../src")

import warnings
warnings.filterwarnings("ignore")

import json
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm

In [3]:
from Config.config import PATHS
from Classes.parameter_recovery import ParameterFit
from Classes.cognitive_model_agents import MODELS, M1_MODELS, M2_MODELS, M3_MODELS

## Fit to 2-player data

In [4]:
#Load 2P data
data_folder = PATHS['human_data']
file_name = '2-player-UR.csv'
file = data_folder / Path(file_name)
print(f'Loading data from {file}...')
data = pd.read_csv(file)
data.head(2)

Loading data from /Users/edgar/Documents/GitHub/Cognitive_Models_El_Farol_Bar_Problem/data/human/2-player-UR.csv...


,source,treatment,group,round,player,choice,score,attendance,ac_score,threshold,num_players,room,decision
0,human,score + grid,Grupo-0001,1,405186428721292,0,0,1,25,0.5,2,Grupo-0001,0
1,human,score + grid,Grupo-0001,1,728608683559397,1,1,1,25,0.5,2,Grupo-0001,1


In [20]:
# check_out_these = ['Attendance-M2', 'MFP-M2', 'Payoff-M2']
# check_out_these = ['Fairness-M2']
# check_out_these = ['FRA']
# check_out_these = ['Payoff-M2', 'Fairness-M2', 'AvailableSpace-M2']
# check_out_these = ['Payoff-M2', 'Fairness-M2']
# my_models = [model for model in MODELS if model.name() in check_out_these]
my_models = MODELS

Fit with scipy

In [21]:
ParameterFit.run(
    data=data,
    model_list=my_models,
    best_fit_path=PATHS['parameter_fit_results'] / 'best_fit_2P_Scipy.json',
    optimizer_type='scipy',
    hyperparameters=None,
    new_file=True
)

Fitting models...:   0%|                                 | 0/23 [00:00<?, ?it/s]

Fitting data to model Priors-M1...
Creating parameter recovery class...
Running optimizer...
Finding deviance for 2 players and threshold 0.5...


Fitting models...:   4%|█                        | 1/23 [00:13<04:46, 13.04s/it]

Optimal parameters for 2 players and threshold 0.5:
{'inverse_temperature': np.float64(44.402932455142974), '0-go_prob_0': np.float64(0.6042590731270663), '1-go_prob_0': np.float64(0.535936097600243)}
Deviance: -3133.2871612081335
AIC: 6272.574322416267
--------------------------------------------------
Fitting data to model Priors-M2...
Creating parameter recovery class...
Running optimizer...
Finding deviance for 2 players and threshold 0.5...


Fitting models...:   9%|██▏                      | 2/23 [01:24<16:39, 47.62s/it]

...

Optimal parameters for 2 players and threshold 0.5:
{'inverse_temperature': np.float64(8.91756753828036), 'belief_strength': np.float64(17.996949469012023)}
Deviance: -2560.924508920179
AIC: 5125.849017840358
--------------------------------------------------
Fitting data to model FRA...
Creating parameter recovery class...
Running optimizer...
Finding deviance for 2 players and threshold 0.5...


Fitting models...:  96%|██████████████████████▉ | 22/23 [11:05<00:22, 22.82s/it]

Optimal parameters for 2 players and threshold 0.5:
{'inverse_temperature': np.float64(45.4918893684743), 'len_history': np.float64(1.0874576255819515), 'max_regions': np.float64(5.637494510964563)}
Deviance: -7004.424310430994
AIC: 14014.848620861989
--------------------------------------------------
Fitting data to model FRA+Payoff+Attendance...
Creating parameter recovery class...
Running optimizer...
Finding deviance for 2 players and threshold 0.5...


Fitting models...: 100%|████████████████████████| 23/23 [12:05<00:00, 31.54s/it]

Optimal parameters for 2 players and threshold 0.5:
{'inverse_temperature': np.float64(6.384494971143151), 'bias': np.float64(0.7478171885358551), 'learning_rate': np.float64(0.19868152793467692), 'len_history': np.float64(1.027463686100999), 'max_regions': np.float64(1.1292748335562928), 'delta': np.float64(0.054266539488045204)}
Deviance: -2223.0433425112005
AIC: 4458.086685022401
--------------------------------------------------


Fit with Bayesian Optimizer

In [22]:
hyperparameters = {
    'init_points':128,
    'n_iter':64
}

ParameterFit.run(
    data=data,
    model_list=my_models,
    best_fit_path=PATHS['parameter_fit_results'] / 'best_fit_2P_Bayesian.json',
    optimizer_type='bayesian',
    hyperparameters=hyperparameters,
    new_file=True
)

Fitting models...:   0%|                                 | 0/23 [00:00<?, ?it/s]

Fitting data to model Priors-M1...
Creating parameter recovery class...
Running optimizer...
Finding deviance for 2 players and threshold 0.5...
|   iter    |  target   | invers... | 0-go_p... | 1-go_p... |
-------------------------------------------------------------
| 1         | -9992.713 | 27.272386 | 0.7203244 | 0.0001143 |
| 2         | -5798.146 | 20.046952 | 0.1467558 | 0.0923385 |
| 3         | -3534.584 | 12.734393 | 0.3455607 | 0.3967674 |
| 4         | -3398.272 | 34.945454 | 0.4191945 | 0.6852195 |
| 5         | -6493.211 | 13.880491 | 0.8781174 | 0.0273875 |
| 6         | -3294.720 | 43.239453 | 0.4173048 | 0.5586898 |
| 7         | -4421.551 | 9.8443771 | 0.1981014 | 0.8007445 |
| 8         | -3656.494 | 62.000479 | 0.3134241 | 0.6923226 |
| 9         | -5292.692 | 56.212516 | 0.8946066 | 0.0850442 |
| 10        | -5003.261 | 3.4604513 | 0.1698304 | 0.8781425 |
| 11        | -5094.453 | 7.1958505 | 0.4211076 | 0.9578895 |
| 12        | -3405.583 | 34.589412 | 0.6918771 |

Fitting models...:   4%|█                        | 1/23 [01:18<28:56, 78.93s/it]

| 192       | -4925.560 | 34.599226 | 0.9620220 | 0.7823591 |
Optimal parameters for 2 players and threshold 0.5:
{'inverse_temperature': np.float64(17.571710519428116), '0-go_prob_0': np.float64(0.6039313617062868), '1-go_prob_0': np.float64(0.5488880161458932)}
Deviance: -3134.0499791187362
AIC: 6274.0999582374725
--------------------------------------------------
Fitting data to model Priors-M2...
Creating parameter recovery class...
Running optimizer...
Finding deviance for 2 players and threshold 0.5...
|   iter    |  target   | invers... | 0-go_p... | 0-go_p... | 0-go_p... | 0-go_p... | 0-go_p... | 0-go_p... | 1-go_p... | 1-go_p... | 1-go_p... | 1-go_p... | 1-go_p... | 1-go_p... |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
| 1         | -7037.238 | 27.272386 | 0.7203244 | 0.0001143 | 0.3023325 | 0.1467558 | 0.0923385 | 0.1862602 | 0.3455607 | 

Fitting models...:   9%|██▏                      | 2/23 [02:57<31:45, 90.76s/it]

...

| 192       | -2564.791 | 9.9817232 | 18.343503 |
Optimal parameters for 2 players and threshold 0.5:
{'inverse_temperature': np.float64(9.844377131499728), 'belief_strength': np.float64(20.612047419403)}
Deviance: -2560.9837934867696
AIC: 5125.967586973539
--------------------------------------------------
Fitting data to model FRA...
Creating parameter recovery class...
Running optimizer...
Finding deviance for 2 players and threshold 0.5...
|   iter    |  target   | invers... | len_hi... | max_re... |
-------------------------------------------------------------
| 1         | -12334.67 | 27.272386 | 1.7203244 | 1.0010293 |
| 2         | -12334.67 | 20.046952 | 1.1467558 | 1.8310473 |
| 3         | -6629.915 | 12.734393 | 1.3455607 | 4.5709072 |
| 4         | -7004.424 | 34.945454 | 1.4191945 | 7.1669755 |
| 5         | -12334.67 | 13.880491 | 1.8781174 | 1.2464883 |
| 6         | -7004.424 | 43.239453 | 1.4173048 | 6.0282084 |
| 7         | -5632.125 | 9.8443771 | 1.1981014 | 8.2067

Fitting models...:  96%|██████████████████████▉ | 22/23 [31:02<01:24, 84.97s/it]

| 192       | -2716.550 | 1.0       | 2.0       | 7.1550642 |
Optimal parameters for 2 players and threshold 0.5:
{'inverse_temperature': np.float64(1.0), 'len_history': np.float64(2.0), 'max_regions': np.float64(6.403345920781902)}
Deviance: -2716.550246457715
AIC: 5439.10049291543
--------------------------------------------------
Fitting data to model FRA+Payoff+Attendance...
Creating parameter recovery class...
Running optimizer...
Finding deviance for 2 players and threshold 0.5...
|   iter    |  target   | invers... |   bias    | learni... | len_hi... | max_re... |   delta   |
-------------------------------------------------------------------------------------------------
| 1         | -3144.443 | 27.272386 | 0.7203244 | 0.0001143 | 1.3023325 | 2.3208030 | 0.0092338 |
| 2         | -4445.302 | 12.734393 | 0.3455607 | 0.3967674 | 1.5388167 | 4.7727506 | 0.0685219 |
| 3         | -2386.740 | 13.880491 | 0.8781174 | 0.0273875 | 1.6704675 | 4.7557432 | 0.0558689 |
| 4         | -696

Fitting models...: 100%|████████████████████████| 23/23 [32:55<00:00, 85.90s/it]

| 192       | -8408.810 | 10.009026 | 0.0       | 1.0       | 2.0       | 6.7180449 | 0.0       |
Optimal parameters for 2 players and threshold 0.5:
{'inverse_temperature': np.float64(3.871924635666916), 'bias': np.float64(0.7003112406431856), 'learning_rate': np.float64(0.5392416719518817), 'len_history': np.float64(1.4465769849601595), 'max_regions': np.float64(4.123890922501882), 'delta': np.float64(0.05049258147684683)}
Deviance: -2356.3586467859277
AIC: 4724.717293571855
--------------------------------------------------


Keep best fit from both optimizers

In [24]:
df_2P_Scipy = pd.read_json(PATHS['parameter_fit_results'] / 'best_fit_2P_Scipy.json', lines=True)
df_2P_Bayesian = pd.read_json(PATHS['parameter_fit_results'] / 'best_fit_2P_Bayesian.json', lines=True)

df = pd.merge(left=df_2P_Scipy, right=df_2P_Bayesian, on='model_name', how='outer')

df_list = []
for model in my_models:
    df1 = df[df['model_name'] == model.name()]
    deviance_x = -df1['deviance_x'].unique()[0]
    deviance_y = -df1['deviance_y'].unique()[0]
    if deviance_x < deviance_y:
        df1.drop(columns=[col for col in df1.columns if '_y' in col], inplace=True)
        df1.rename(columns={col: col.replace('_x', '') for col in df1.columns}, inplace=True)
    else:
        df1.drop(columns=[col for col in df1.columns if '_x' in col], inplace=True)
        df1.rename(columns={col: col.replace('_y', '') for col in df1.columns}, inplace=True)
    df_list.append(df1)
df = pd.concat(df_list, ignore_index=True)
df.to_json(PATHS['parameter_fit_results'] / 'best_fit_2P.json', orient='records', lines=True)




## Fit to all player data

In [ ]:
# Load data into a dataframe
# file_names = [
# 	# '2-player-UR.csv',
# 	'3-player-IU.csv',
# 	'4-player-IU.csv',
# 	'5-player-IU.csv',
# 	'6-player-IU.csv',
# 	'7-player-IU.csv',
# 	'8-player-IU.csv',
# 	'9-player-IU.csv',
# 	'11-player-IU.csv',
# 	'12-player-IU.csv',
# ]
file_names = ['multi-player.csv']
data_folder = PATHS['human_data']
df_list = list()
for file_name in file_names:
	file = data_folder / Path(file_name)
	print(f'Loading data from {file}...')
	df = pd.read_csv(file)
	df_list.append(df)
data = pd.concat(df_list, ignore_index=True)
columns = ['threshold', 'num_players', 'group', 'round', 'player', 'score', 'decision']
drop_columns = [col for col in data.columns if col not in columns]
data.drop(columns=drop_columns, inplace=True)
data.head()

Loading data from /Users/edgar/Documents/GitHub/Cognitive_Models_El_Farol_Bar_Problem/data/human/3-player-IU.csv...
Loading data from /Users/edgar/Documents/GitHub/Cognitive_Models_El_Farol_Bar_Problem/data/human/4-player-IU.csv...
Loading data from /Users/edgar/Documents/GitHub/Cognitive_Models_El_Farol_Bar_Problem/data/human/5-player-IU.csv...
Loading data from /Users/edgar/Documents/GitHub/Cognitive_Models_El_Farol_Bar_Problem/data/human/6-player-IU.csv...
Loading data from /Users/edgar/Documents/GitHub/Cognitive_Models_El_Farol_Bar_Problem/data/human/7-player-IU.csv...
Loading data from /Users/edgar/Documents/GitHub/Cognitive_Models_El_Farol_Bar_Problem/data/human/8-player-IU.csv...
Loading data from /Users/edgar/Documents/GitHub/Cognitive_Models_El_Farol_Bar_Problem/data/human/9-player-IU.csv...
Loading data from /Users/edgar/Documents/GitHub/Cognitive_Models_El_Farol_Bar_Problem/data/human/11-player-IU.csv...
Loading data from /Users/edgar/Documents/GitHub/Cognitive_Models_El_Far

,threshold,round,player,decision,score,num_players,group
0,0.333333,1.0,406360458182849,1.0,-1.0,3,17
1,0.333333,1.0,679877000681823,1.0,-1.0,3,17
2,0.333333,1.0,740425726975861,1.0,-1.0,3,17
3,0.333333,2.0,406360458182849,1.0,-1.0,3,17
4,0.333333,2.0,679877000681823,1.0,-1.0,3,17


In [37]:
# check_out_these = ['Attendance-M2', 'MFP-M2', 'Payoff-M2']
# check_out_these = ['Fairness-M2']
# check_out_these = ['FRA']
# check_out_these = ['Payoff-M2', 'Fairness-M2', 'AvailableSpace-M2']
# check_out_these = ['Payoff-M2', 'Fairness-M2']
# my_models = [model for model in MODELS if model.name() in check_out_these]
my_models = MODELS

Fit with scipy

In [38]:
ParameterFit.run(
    data=data,
    model_list=my_models,
    best_fit_path=PATHS['parameter_fit_results'] / 'best_fit_all_Scipy.json',
    optimizer_type='scipy',
    hyperparameters=None,
    new_file=True
)

Fitting models...:   0%|                                 | 0/23 [00:00<?, ?it/s]

Fitting data to model Priors-M1...
Creating parameter recovery class...
Running optimizer...
Finding deviance for 4 players and threshold 0.25...


Fitting models...:   0%|                                 | 0/23 [00:00<?, ?it/s]

Error with state 0
self.shape=(1,)
self.states=[0]
self.go_prob.shape=(1,)
self.go_prob=array([0.])
Check also possible error with free parameters:
my_free_parameters={'inverse_temperature': np.float64(40.77687544460334)}


Exception: 'go_prob_0'

Fit with Bayesian Optimizer

In [ ]:
hyperparameters = {
    'init_points':128,
    'n_iter':64
}

ParameterFit.run(
    data=data,
    model_list=my_models,
    best_fit_path=PATHS['parameter_fit_results'] / 'best_fit_all_Bayesian.json',
    optimizer_type='bayesian',
    hyperparameters=hyperparameters,
    new_file=True
)